In [2]:
## Py version 3.7 

In [5]:
from utils import *
from functions import *

### In this notebook I'll scrape data based on categoy news in Wolof.
- Each category news has multiple article
- Each article has title, text, and tag (category of news)


We'll start by pulling all links in each `category`

## 1.0 Crawler and Scraper setup

In [6]:
base_url = 'https://saabal.com'
def parse_url(url):
    page = requests.get(url).text
    return bs(page)

# 1. Links crawler

all_links = []  
# Récup liens de categories
def get_urls(base_url):
    # category_page = requests.get(base_url).text
    # category_page_soup = bs(category_page)
    cateries_news = [a.text for a in bs(requests.get(base_url).text).find(class_="widget clearfix widget_categories").find_all("a")]
    liens_categories = [a['href'] for a in bs(requests.get(base_url).text).find(class_="widget clearfix widget_categories").find_all("a")]
    # Let's grap all link for all categories news
    for lien in liens_categories:
        page = parse_url(lien)
        for soup in page.find_all(class_="entry-title archive-entry-title mt-0"):
            link = soup.find('a')['href']
            all_links.append(link)
    return cateries_news, liens_categories, all_links

cateries_news, liens_categories, links = get_urls(base_url)

links[:10]

['https://saabal.com/mali-raxas-gaa-ngay-wey/',
 'https://saabal.com/njiiti-reew-yu-mbootaayug-cedeao-mbootaay-gi-emb-reewi-afrig-sowu-jant-yi-bennoo-ci-wallug-koppar/',
 'https://saabal.com/gis-gis-ca-la-xew-mali/',
 'https://saabal.com/bomb-nikleer-ca-hirosimaa-widewoo/',
 'https://saabal.com/boomug-jullit-na-ca-yugooslawi/',
 'https://saabal.com/burkina-faaso-xiixaankat-ya-ray-nanu-fa-njiitu-gox-bu-pensa-ak/',
 'https://saabal.com/ndax-yalla-moo-nu-teg-nattu-walla-sunug-nangoo-nuy-lor-mengaleel-dundug-doomi-reewum-libaan-ci-senegaal-ak-dundug-doomi-senegaal-ca-libaan/',
 'https://saabal.com/liy-firndeel-ne-yaqub-lakku-wolof-a-gen-a-ganjaru-bob-faranse/',
 'https://saabal.com/tenub-lewopool-ii-di-raykatu-nit-bu-mag-jot-na-wallam/',
 'https://saabal.com/xoolal-rekk-ni-ko-alkaati-yi-raye%f0%9f%98%a2/']

#### Parcours de chaque catégorie, tous ces articles 

In [7]:
titles = []
link_to_content = []
textes = []
categories = []

def get_data(urls):
    '''
    input: links of liens_categories
    Output: dictionary of title and link to content article
    '''
    for url in urls:
        
        page = parse_url(url)
        for soup in page.find_all(class_="entry-title archive-entry-title mt-0"):
            categorie = url.split('/')[-2]
            t = soup.text.strip()
            l = soup.find('a')['href']
            titles.append(t)
            link_to_content.append(l)
            # get text data
            content = parse_url(l)
            categories.append(categorie)
            texte = [a.text for a in content.find(class_ = "entry-content").find_all('p') if a]
            textes.append(texte)

    return titles, link_to_content, textes, categories # A arranger

In [8]:
titles, link_to_content, textes, categories = get_data(liens_categories)

# to dic then to df
data = {}
data['titles'], data['link_to_content'], data['textes'], data['categories'] = get_data(liens_categories)
df = pd.DataFrame(data)
df.sample(25)

,titles,link_to_content,textes,categories
168,Jafe-jafe yi ci sëy (Alsaan Jóob) – Xaaj 1,https://saabal.com/jafe-jafe-yi-ci-sey-alsaan-...,"[Alsaan Jóob , Amoon na fi, booba ak leegi mi ...",chronique
225,DOOMU SENEGAAL BU NJËKK BI AM JÀNGORO JI,https://saabal.com/doomu-senegaal-bu-njekk-bi-...,[Benn Móodu-Móodu bu dëkk Tuubaa moo am jàngor...,xalaat
67,Bu doon yaw nan ngay def?,https://saabal.com/bu-doon-yaw-nan-ngay-def/,"[, Benn xale bu góor bu jàng ba am ay lijaasaa...",non-classe
34,Sunu maam yi ñoo jàngal tubaab yi bind …,https://saabal.com/sunu-maam-yi-noo-jangal-tub...,"[« Bu ag lóofóon yamee, dëgg feeñ. Lóofoonulee...",mboor
188,Waxtaan wu am solo – Maysa Maaraa di ganug Saa...,https://saabal.com/waxtaan-wu-am-solo-maysa-ma...,"[Waxtaan wu Maysa Maaraa séq ak Saabal.com, gi...",non-classe
2,Gis-gis ca la xew Mali,https://saabal.com/gis-gis-ca-la-xew-mali/,[Ci fan yii ci weeru ut nga xam ne ci lañu jël...,bitim-reew
227,Xadi Maróon: dee ak doom ja ndax ñàkk fajuwaay...,https://saabal.com/xadi-maroon-dee-ak-doom-ja-...,[Ña dëkke ca Aaga nekk ca diggante Caajaay ak ...,xew-xew-jamono
216,"Toltolaan ci sa Tooli maam, pàcc 2:",https://saabal.com/toltolaan-ci-sa-tooli-maam-...,[Du moom rekk nag daanaka yeneen njaatige yépp...,xalaat
73,Kujje gi ci diggante tarixa yi ci Senegaal (Al...,https://saabal.com/kujje-gi-ci-diggante-tarixa...,"[, Noonu laa daaldi dindee jëfekaayu Facebook ...",politig
111,Jur doom ray !,https://saabal.com/jur-doom-ray/,[Démb mbir mu ni mel lees àtte ca àttewaayu pé...,xew-xew-jamono


## 2.0 Prepocessing
Words tonkenizing

In [15]:
df.to_csv('data/raw_data.csv') # temps save

In [10]:
# Split articles text into each row in textes' column by sentence and store each sentence to new row. 

# init 
sentences = []
categories = []
titles = []
link_to_content = []
ids = []

for idx, row in df.iterrows():
    categorie = row.categories
    title = row.titles
    link_to = row.link_to_content
    identifier = idx
    for sentence in row.textes:
        
        categories.append(categorie)
        sentences.append(sentence)
        ids.append(identifier)
        titles.append(title)
        
        link_to_content.append(link_to)
        
        
textes_to_sentences = pd.DataFrame({"id": ids, "titles": titles,
                                      "sentences": sentences,
                                      "categories": categories, "link_to_content": link_to_content})

In [11]:
textes_to_sentences

,id,titles,sentences,categories,link_to_content
0,0,Ndax Yàlla moo nu teg nattu walla sunug nangoo...,"Partager la publication ""Ndax Yàlla moo nu teg...",bitim-reew,https://saabal.com/ndax-yalla-moo-nu-teg-nattu...
1,1,Liy firndeel ne yaqub làkku Wolof a gën a gànj...,"Bismillaah, ci turu Yàlla, ma tàmbali. Alhamdu...",bitim-reew,https://saabal.com/liy-firndeel-ne-yaqub-lakku...
2,1,Liy firndeel ne yaqub làkku Wolof a gën a gànj...,Bokk yi kaayleen nu waxtaan ci làkku farañse!📢,bitim-reew,https://saabal.com/liy-firndeel-ne-yaqub-lakku...
3,1,Liy firndeel ne yaqub làkku Wolof a gën a gànj...,Mbokk mbaa xam nga ne làkku farañse mi nga naw...,bitim-reew,https://saabal.com/liy-firndeel-ne-yaqub-lakku...
4,1,Liy firndeel ne yaqub làkku Wolof a gën a gànj...,"Topp ma, dinga ci xam dara👊🏿!",bitim-reew,https://saabal.com/liy-firndeel-ne-yaqub-lakku...
5,1,Liy firndeel ne yaqub làkku Wolof a gën a gànj...,Ci farañse:,bitim-reew,https://saabal.com/liy-firndeel-ne-yaqub-lakku...
6,1,Liy firndeel ne yaqub làkku Wolof a gën a gànj...,"😌-Waati xam-xam yi, waati gereg ak yu latiŋ yu...",bitim-reew,https://saabal.com/liy-firndeel-ne-yaqub-lakku...
7,1,Liy firndeel ne yaqub làkku Wolof a gën a gànj...,Niral : hydro-phile : nob-ndox🤨 lay tekki ci g...,bitim-reew,https://saabal.com/liy-firndeel-ne-yaqub-lakku...
8,1,Liy firndeel ne yaqub làkku Wolof a gën a gànj...,Yeneen waat yi noonu lañu mel.,bitim-reew,https://saabal.com/liy-firndeel-ne-yaqub-lakku...
9,1,Liy firndeel ne yaqub làkku Wolof a gën a gànj...,😌-Waat yi jëm ci xare yu bari ci Almaa lañu juge,bitim-reew,https://saabal.com/liy-firndeel-ne-yaqub-lakku...


In [19]:
textes_to_sentences.to_csv('data/raw_data_splited.csv') # temp save

## 2.1 Data cleaning task.